In [1]:
import meanderpy as mp
import matplotlib.pyplot as plt
import numpy as np
%matplotlib qt

In [36]:
from importlib import reload
reload(mp)

<module 'meanderpy' from '/Users/zoltan/Dropbox/Channels/meanderpy/meanderpy/meanderpy.py'>

## Input parameters

In [37]:
W = 200.0                    # channel width (m)
D = 12.0                     # channel depth (m)
pad = 100                    # padding (number of nodepoints along centerline)
deltas = 50.0                # sampling distance along centerline
nit = 2000                   # number of iterations
Cf = 0.022                    # dimensionless Chezy friction factor
crdist = 1.5*W               # threshold distance at which cutoffs occur
kl = 60.0/(365*24*60*60.0)   # migration rate constant (m/s)
kv =  1.0E-11               # vertical slope-dependent erosion rate constant (m/s)
dt = 2*0.05*365*24*60*60.0     # time step (s)
dens = 1000                  # density of water (kg/m3)
saved_ts = 20                # which time steps will be saved
n_bends = 30                 # approximate number of bends you want to model
Sl = 0.0                     # initial slope (matters more for submarine channels than rivers)
t1 = 500                    # time step when incision starts
t2 = 700                    # time step when lateral migration starts
t3 = 1400                    # time step when aggradation starts
aggr_factor = 4e-9           # aggradation factor (m/s, about 0.18 m/year, it kicks in after t3)

## Initialize model

In [38]:
ch = mp.generate_initial_channel(W,D,Sl,deltas,pad,n_bends) # initialize channel
chb = mp.ChannelBelt(channels=[ch],cutoffs=[],cl_times=[0.0],cutoff_times=[]) # create channel belt object

## Run simulation

In [40]:
chb.migrate(nit,saved_ts,deltas,pad,crdist,Cf,kl,kv,dt,dens,t1,t2,t3,aggr_factor) # channel migration
fig = chb.plot('strat', 20, 60, chb.cl_times[-1], len(chb.channels)) # plotting

100%|██████████| 2000/2000 [00:57<00:00, 34.89it/s]


In [43]:
fig = chb.plot('morph', 20, 60, chb.cl_times[-1], len(chb.channels)) # plotting

## Create movie

In [44]:
dirname = '/Users/zoltan/Dropbox/Channels/temp/'
chb.create_movie(xmin=10000, xmax=30000, plot_type='strat', filename='movie', dirname=dirname,
                 pb_age = 1, ob_age = 20, end_time = chb.cl_times[-1], n_channels = len(chb.channels))

## Build 3D fluvial model

In [45]:
h_mud = 0.3 # thickness of overbank deposit for each time step
dx = 20.0 # gridcell size in meters

chb_3d, xmin, xmax, ymin, ymax = chb.build_3d_model('fluvial',h_mud=h_mud,levee_width=800.0,h=12.0,w=W,bth=0.0,
                            dcr=10.0,dx=dx,delta_s=deltas,starttime=chb.cl_times[0],endtime=chb.cl_times[-1],
                            xmin=0,xmax=0,ymin=0,ymax=0)

100%|██████████| 101/101 [00:05<00:00, 17.76it/s]


In [47]:
# create plots
fig1,fig2,fig3 = chb_3d.plot_xsection(100, [[0.5,0.25,0],[0.9,0.9,0],[0.5,0.25,0]], 4)

## Build 3D submarine channel model

In [48]:
W = 200.0                    # channel width (m)
D = 12.0                     # channel depth (m)
pad = 50                    # padding (number of nodepoints along centerline)
deltas = 100.0                # sampling distance along centerline
nit = 1500                   # number of iterations
Cf = 0.02                    # dimensionless Chezy friction factor
crdist = 1.5*W               # threshold distance at which cutoffs occur
kl = 60.0/(365*24*60*60.0)   # migration rate constant (m/s)
kv =  1.0E-11               # vertical slope-dependent erosion rate constant (m/s)
dt = 2*0.05*365*24*60*60.0     # time step (s)
dens = 1000                  # density of water (kg/m3)
saved_ts = 20                # which time steps will be saved
n_bends = 50                 # approximate number of bends you want to model
Sl = 0.01                     # initial slope (matters more for submarine channels than rivers)
t1 = 500                    # time step when incision starts
t2 = 700                    # time step when lateral migration starts
t3 = 1000                    # time step when aggradation starts
aggr_factor = 4.0          # aggradation factor (it kicks in after t3)

In [49]:
ch = mp.generate_initial_channel(W,D,Sl,deltas,pad,n_bends) # initialize channel
chb = mp.ChannelBelt(channels=[ch],cutoffs=[],cl_times=[0.0],cutoff_times=[]) # create channel belt object

In [50]:
chb.migrate(nit,saved_ts,deltas,pad,crdist,Cf,kl,kv,dt,dens,t1,t2,t3,aggr_factor) # channel migration
fig = chb.plot('strat',20,60,chb.cl_times[-1],len(chb.channels)) # plotting

100%|██████████| 1500/1500 [00:18<00:00, 81.94it/s] 


In [51]:
h_mud = 3.0*np.ones((len(chb.cl_times[20:]),))
dx = 15.0

chb_3d, xmin, xmax, ymin, ymax = chb.build_3d_model('submarine',h_mud=h_mud,levee_width=5000.0,h=12.0,w=W,bth=6.0,
                            dcr=7.0,dx=dx,delta_s=deltas,starttime=chb.cl_times[20],endtime=chb.cl_times[-1],
                            xmin=0,xmax=0,ymin=0,ymax=0)


100%|██████████| 56/56 [00:02<00:00, 19.68it/s]


In [52]:
fig1,fig2,fig3 = chb_3d.plot_xsection(100, [[0.5,0.25,0],[0.9,0.9,0],[0.5,0.25,0]], 10)